In [1]:
import torch
from torch.utils.data import DataLoader
from object_detection_data import ObjectDetectionDataset, Encoder, TransformedObjectDetectionDataset
import torch.nn as nn
from pytorch_lightning.callbacks import RichProgressBar
import pytorch_lightning as pl
import torch.nn.functional as F
import torch.optim as optim
from object_detection_models import LightningMultiClassJetNet
from pytorch_lightning.callbacks import Timer
import torchvision.transforms as T
torch.manual_seed(2)


In [2]:
torch.cuda.device_count()
from object_detection_models import MultiClassJetNet

In [3]:
default_scalings = torch.tensor(
    [[0.25, 0.25], [0.2, 0.2], [0.125, 0.125], [0.125 / 2, 0.125 / 2], [0.125 / 4, 0.125 / 4]])
feature_map_size = (8, 10)
num_classes = 4
encoder = Encoder(default_scalings, feature_map_size, num_classes)
data_path = 'data/train'
# transformed_train_data = TransformedObjectDetectionDataset(
#     'data/train', encoder)
# transformed_val_data = TransformedObjectDetectionDataset(
#     'data/val', encoder)

transforms = T.Compose([T.Grayscale(), T.PILToTensor(), T.ConvertImageDtype(
    torch.float32), T.Resize((60, 80))])
transformed_train_data = ObjectDetectionDataset(
    'data/train', encoder, transforms=transforms)


In [6]:
train_loader = DataLoader(transformed_train_data, batch_size=8,
                          shuffle=True, num_workers=0)
# val_loader = DataLoader(transformed_val_data, batch_size=8,
#                         shuffle=False, num_workers=0)
pl_model = LightningMultiClassJetNet(num_classes, len(default_scalings))
# pl_model.model = torch.compile(MultiClassJetNet(num_classes, len(default_scalings)))
# trainer = pl.Trainer(limit_predict_batches=100,
#                      max_epochs=20, callbacks=RichProgressBar(), accelerator='gpu', devices=[0,1])

trainer = pl.Trainer(limit_predict_batches=100,
                     max_epochs=20, callbacks=RichProgressBar())
trainer.fit(model=pl_model, train_dataloaders=train_loader)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/jonathan/hulks/ml/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:108: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(


┏━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name  ┃ Type             ┃ Params ┃
┡━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ model │ MultiClassJetNet │ 27.4 K │
└───┴───────┴──────────────────┴────────┘

Trainable params: 27.4 K                                                                                           
Non-trainable params: 0                                                                                            
Total params: 27.4 K                                                                                               
Total estimated model params size (MB): 0

/home/jonathan/hulks/ml/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Output()